**Download Huggingface Dataset Handler**

In [ ]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 41.7 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 243 kB 47.5 MB/s 
     |████████████████████████████████| 132 kB 45.7 MB/s 
     |████████████████████████████████| 192 kB 38.0 MB/s 
     |████████████████████████████████| 271 kB 67.6 MB/s 
     |████████████████████████████████| 160 kB 60.1 MB/s 


**Imports**

In [ ]:
import PIL
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from scipy.spatial import distance
import numpy as np

import cv2
from google.colab.patches import cv2_imshow
import matplotlib.image as pltimg
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

**Download CIFAR 100 TEST dat**a

In [ ]:
from datasets import load_dataset
dataset = load_dataset('cifar100', split='test')

Downloading:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/169M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cifar100 downloaded and prepared to /root/.cache/huggingface/datasets/cifar100/cifar100/1.0.0/f46fc68a3c4e336557d8541eb991accda01231b257aac9e56c1d3b48102fc780. Subsequent calls will reuse this data.


# **RGB Computations**

**Allocate locations of images from test data**

In [ ]:
'''
Index - Class from data: 
0 Apple ,
53 Orange, 
22 Clock,
telephone 86, 
television 87 
Spider 79, 
man 46, 
lobster 45, 
boy 11
'''
# Store the locations of the images in lists
apple_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 0]
orange_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 53]
clock_locs = [i for i,v in enumerate(dataset['fine_label']) if v ==22]
telephone_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 86]
television_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 87]
spider_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 79]
man_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 46]
lobster_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 45]
boy_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 11]


**Utility functions for RGB - Eculidean Computations**

In [ ]:
# Compute mean RGB from the images
def get_mean_rgb(itemloclist):
    means = [] # Store means of each images
    # Load images from list, calculate width height, get RGB colors.
    for loc in itemloclist:
        PIL_image = Image.fromarray(np.array(dataset['img'][loc]).astype('uint8'), 'RGB')#.resize((128,128))
        w,h = PIL_image.size
        colors = PIL_image.getcolors(w*h)
        mean = [0, 0, 0]
        counter = 0
        # result = sorted(colors, key=lambda tup: tup[0], reverse=True)
        
        # From each image, add all Rs, Gs, Bs to list. At the end of loading compute mean for that image and append to means.
        for res in colors:
            #red = res[0][0]
            mean[0] += res[1][0]
            #green = res[0][1]
            mean[1] += res[1][1]
            #blue = res[0][2]
            mean[2] += res[1][2]
            counter +=1
        means.append([x/(counter+0.0001) for x in mean]) # +0.0001 to avoid division by 0
    return means


# Compute euclidean distances of mean RGB to target colors: Red, Green, Blue, Orange, and Black
def get_euc_dist(meanslist):
  euc_dist_r = []     # Euclidean Distance of Sample to Red Color
  euc_dist_g = []     # Euclidean Distance of Sample to Green Color
  euc_dist_b = []     # Euclidean Distance of Sample to Blue Color
  euc_dist_o = []     # Euclidean Distance of Sample to Orange Color
  euc_dist_bl = []    # Euclidean Distance of Sample to Black Color
  for datapoints in meanslist:
    xs = datapoints[0]
    ys = datapoints[1] 
    zs = datapoints[2]
    euc_dist_r.append(distance.euclidean([xs,ys,zs], [255, 0, 0]))
    euc_dist_g.append(distance.euclidean([xs,ys,zs], [0, 255, 0]))
    euc_dist_b.append(distance.euclidean([xs,ys,zs], [0, 0, 255]))
    euc_dist_o.append(distance.euclidean([xs,ys,zs], [255, 165, 0]))
    euc_dist_bl.append(distance.euclidean([xs,ys,zs], [0, 0, 0]))
  return {"euc_dist_r":euc_dist_r, 
  "euc_dist_g":euc_dist_g,
  "euc_dist_b":euc_dist_b,
  "euc_dist_o":euc_dist_o,
  "euc_dist_bl":euc_dist_bl}

# Calculate the overall mean of the euclidean distances.
def get_mean_colors_from_all_images(dists):
  return np.array(dists['euc_dist_r']).mean(), np.array(dists['euc_dist_g']).mean() ,np.array(dists['euc_dist_b']).mean(),np.array(dists['euc_dist_o']).mean(),np.array(dists['euc_dist_bl']).mean()


**Compute Mean RGBs, Euclidean Distances, Mean Euclidean Distances**

In [ ]:
# For Apple
apple_means = get_mean_rgb(apple_locs)
apple_dists = get_euc_dist(apple_means)
apple_col_means = get_mean_colors_from_all_images(apple_dists)
print('Apple Done')
# For Orange
orange_means = get_mean_rgb(orange_locs)
orange_dists = get_euc_dist(orange_means)
orange_col_means = get_mean_colors_from_all_images(orange_dists)
print('Orange Done')
# For Clock
clock_means = get_mean_rgb(clock_locs)
clock_dists = get_euc_dist(clock_means)
clock_col_means = get_mean_colors_from_all_images(clock_dists)
print('Clock Done')
# For Telephone
telephone_means = get_mean_rgb(telephone_locs)
telephone_dists = get_euc_dist(telephone_means)
telephone_col_means = get_mean_colors_from_all_images(telephone_dists)
print('Phone Done')
# For Television
television_means = get_mean_rgb(television_locs)
television_dists = get_euc_dist(television_means)
television_col_means = get_mean_colors_from_all_images(television_dists)
print('TV Done')
# For Spider
spider_means = get_mean_rgb(spider_locs)
spider_dists = get_euc_dist(spider_means)
spider_col_means = get_mean_colors_from_all_images(spider_dists)
print('Spider Done')
# For Man
man_means = get_mean_rgb(man_locs)
man_dists = get_euc_dist(man_means)
man_col_means = get_mean_colors_from_all_images(man_dists)
print('Man Done')
# For Lobster
lobster_means = get_mean_rgb(lobster_locs)
lobster_dists = get_euc_dist(lobster_means)
lobster_col_means = get_mean_colors_from_all_images(lobster_dists)
print('Lobster Done')
# For Boy
boy_means = get_mean_rgb(boy_locs)
boy_dists = get_euc_dist(boy_means)
boy_col_means = get_mean_colors_from_all_images(boy_dists)
print('Boy Done')

Apple Done
Orange Done
Clock Done
Phone Done
TV Done
Spider Done
Man Done
Lobster Done
Boy Done


In [ ]:
boy_col_means

(204.99835704081661,
 225.92092045701315,
 235.17839848862488,
 176.13910165332695,
 207.44776526315897)

**Converting Result into CSV**

In [ ]:
def conv2df(means, rd, gd, bd, od,bld,label):
  cls = [label]*len(rd)
  col_data = {'Mean RGB':means,
            'Red':rd,
            'Green':gd,
            'Blue':bd,
            'Orange':od,
            'Black':bld,
            'Class':cls
            }
  return pd.DataFrame.from_dict(col_data)

In [ ]:
apple_df = conv2df(apple_means,
                   apple_dists['euc_dist_r'],
                   apple_dists['euc_dist_g'],
                   apple_dists['euc_dist_b'],
                   apple_dists['euc_dist_o'],
                   apple_dists['euc_dist_bl'],
                   'Apple')
orange_df = conv2df(orange_means,
                   orange_dists['euc_dist_r'],
                   orange_dists['euc_dist_g'],
                   orange_dists['euc_dist_b'],
                   orange_dists['euc_dist_o'],
                   orange_dists['euc_dist_bl'],
                   'Orange')
clock_df = conv2df(clock_means,
                   clock_dists['euc_dist_r'],
                   clock_dists['euc_dist_g'],
                   clock_dists['euc_dist_b'],
                   clock_dists['euc_dist_o'],
                   clock_dists['euc_dist_bl'],
                   'Clock')
telephone_df = conv2df(telephone_means,
                   telephone_dists['euc_dist_r'],
                   telephone_dists['euc_dist_g'],
                   telephone_dists['euc_dist_b'],
                   telephone_dists['euc_dist_o'],
                   telephone_dists['euc_dist_bl'],
                   'Telephone')
television_df = conv2df(television_means,
                   television_dists['euc_dist_r'],
                   television_dists['euc_dist_g'],
                   television_dists['euc_dist_b'],
                   television_dists['euc_dist_o'],
                   television_dists['euc_dist_bl'],
                   'Television')
spider_df = conv2df(spider_means,
                   spider_dists['euc_dist_r'],
                   spider_dists['euc_dist_g'],
                   spider_dists['euc_dist_b'],
                   spider_dists['euc_dist_o'],
                   spider_dists['euc_dist_bl'],
                   'Spider')
man_df = conv2df(man_means,
                   man_dists['euc_dist_r'],
                   man_dists['euc_dist_g'],
                   man_dists['euc_dist_b'],
                   man_dists['euc_dist_o'],
                   man_dists['euc_dist_bl'],
                   'Man')
lobster_df = conv2df(lobster_means,
                   lobster_dists['euc_dist_r'],
                   lobster_dists['euc_dist_g'],
                   lobster_dists['euc_dist_b'],
                   lobster_dists['euc_dist_o'],
                   lobster_dists['euc_dist_bl'],
                   'Lobster')
boy_df = conv2df(boy_means,
                   boy_dists['euc_dist_r'],
                   boy_dists['euc_dist_g'],
                   boy_dists['euc_dist_b'],
                   boy_dists['euc_dist_o'],
                   boy_dists['euc_dist_bl'],
                   'Boy')

In [ ]:
concat_df = pd.concat([apple_df,orange_df,clock_df,telephone_df,television_df,spider_df,man_df,lobster_df,boy_df]).reset_index(drop = True)

In [ ]:
concat_df.to_csv('CIFAR100_COLOR_DIST_NEW.csv',index=False)

# **Edge Computations**

In [ ]:
# Connect to drive to fetch shapes: Circle, Square/Rectangle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Get edges of Target shapes: Circle, Square/Rectangle**

In [ ]:
def getEdgeofShapes(pathToShape):
  img = cv2.imread(pathToShape)
  img = cv2.resize(img, (32, 32))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  edges = cv2.Canny(image=img_gray, threshold1=100, threshold2=200)
  return edges

# Detect edge of circle
edges_circle = getEdgeofShapes('/content/drive/MyDrive/Machine Learning/Zero Shot Learning/NTECH/Image Data/Shapes/circle.png')
# Detect edge of rectangle / square
edges_square = getEdgeofShapes('/content/drive/MyDrive/Machine Learning/Zero Shot Learning/NTECH/Image Data/Shapes/square.png')

**Calculate Minkowski Distances from image edges to target edges**

In [ ]:
def minkowskidist(locs):
  dist_circle = []
  dist_rect = []
  for loc in locs:
    PIL_image = Image.fromarray(np.array(dataset['img'][loc]).astype('uint8'), 'RGB')
    img_gray = cv2.cvtColor(np.float32(PIL_image), cv2.COLOR_BGR2GRAY)
    edges_apple = cv2.Canny(image=np.uint8(img_gray), threshold1=100, threshold2=200)
    
    dist_circle.append(distance.minkowski(edges_circle.flatten(), edges_apple.flatten(), 2))
    dist_rect.append(distance.minkowski(edges_square.flatten(), edges_apple.flatten(), 2))
  return {'circle':dist_circle, 'rect':dist_rect}

In [ ]:
apple_shape_dist = minkowskidist(apple_locs)
orange_shape_dist = minkowskidist(orange_locs)
clock_shape_dist = minkowskidist(clock_locs)
telephone_shape_dist = minkowskidist(telephone_locs)
television_shape_dist = minkowskidist(television_locs)
spider_shape_dist = minkowskidist(spider_locs)
man_shape_dist = minkowskidist(man_locs)
lobster_shape_dist = minkowskidist(lobster_locs)
boy_shape_dist = minkowskidist(boy_locs)

**Convert them into Dataframes to save CSV**

In [ ]:
apple_shape_dist_df = pd.DataFrame.from_dict(apple_shape_dist)
orange_shape_dist_df = pd.DataFrame.from_dict(orange_shape_dist)
clock_shape_dist_df = pd.DataFrame.from_dict(clock_shape_dist)
telephone_shape_dist_df = pd.DataFrame.from_dict(telephone_shape_dist)
television_shape_dist_df = pd.DataFrame.from_dict(television_shape_dist)
spider_shape_dist_df = pd.DataFrame.from_dict(spider_shape_dist)
man_shape_dist_df = pd.DataFrame.from_dict(man_shape_dist)
lobster_shape_dist_df = pd.DataFrame.from_dict(lobster_shape_dist)
boy_shape_dist_df = pd.DataFrame.from_dict(boy_shape_dist)

pd.concat([apple_shape_dist_df,orange_shape_dist_df,clock_shape_dist_df,telephone_shape_dist_df,television_shape_dist_df,spider_shape_dist_df,man_shape_dist_df,lobster_shape_dist_df,boy_shape_dist_df]).reset_index(drop=True).to_csv('Minkowski Dist.csv',index=False)

In [ ]:
print(np.array(apple_shape_dist['circle']).mean(), np.array(apple_shape_dist['rect']).mean())
print(np.array(orange_shape_dist['circle']).mean(), np.array(orange_shape_dist['rect']).mean())
print(np.array(clock_shape_dist['circle']).mean(), np.array(clock_shape_dist['rect']).mean())
print(np.array(telephone_shape_dist['circle']).mean(), np.array(telephone_shape_dist['rect']).mean())
print(np.array(television_shape_dist['circle']).mean(), np.array(television_shape_dist['rect']).mean())
print(np.array(spider_shape_dist['circle']).mean(), np.array(spider_shape_dist['rect']).mean())
print(np.array(man_shape_dist['circle']).mean(), np.array(man_shape_dist['rect']).mean())
print(np.array(lobster_shape_dist['circle']).mean(), np.array(lobster_shape_dist['rect']).mean())
print(np.array(boy_shape_dist['circle']).mean(), np.array(boy_shape_dist['rect']).mean())

2165.3817938915513 2479.8235274733115
2178.329508570902 2482.0783606545
2189.3909984592406 2461.5276511774655
2232.4873857935095 2526.389158841448
2213.148471604447 2457.694282662709
2301.4478454846303 2567.731641731374
2246.33267074079 2503.867269053288
2167.986368348048 2454.3843538300434
2222.824465116193 2475.58971376879


# **Semantic Distance Computations**

**Semantic Vector Computations**

In [ ]:
""" 
###Classes###         
Apple
Orange
Clock
Telephone
Television
Spider
Man
Lobster
Boy

###Semantic Combinations###
Apple - Orange
Apple - Clock
Apple - Telephone
Apple - Television
Apple - Spider
Apple - Man
Apple - Lobster
Apple - Boy

Orange - Clock
Orange - Telephone
Orange - Television
Orange - Spider
Orange - Man
Orange - Lobster
Orange - Boy

Clock - Telephone
Clock - Television
Clock - Spider
Clock - Man
Clock - Lobster
Clock - Boy

Telephone - Television
Telephone - Spider
Telephone - Man
Telephone - Lobster
Telephone - Boy

Television - Spider
Television - Man
Television - Lobster
Television - Boy

Spider - Man
Spider - Lobster
Spider - Boy

Man - Lobster
Man - Boy

Lobster - Boy
"""

**Import Gensim to access Word2Vec (300 Dimension)**

In [ ]:
import gensim.downloader as api
from scipy.spatial import distance
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Returns the corresponding word vector
def retVec(word):
  return wv.get_vector(word)

# Returs euclidean distances between two word vectors
def eucdist(word1,word2):
  return distance.euclidean(retVec(word1), retVec(word2))

# Euc distances before adding extra feats.
print(eucdist('Apple','Orange'))
print(eucdist('Apple','Clock'))
print(eucdist('Apple','Phone'))
print(eucdist('Apple','TV'))
print(eucdist('Apple','Spider'))
print(eucdist('Apple','Man'))
print(eucdist('Apple','Lobster'))
print(eucdist('Apple','Boy'))


print(eucdist('Orange','Clock'))
print(eucdist('Orange','Phone'))
print(eucdist('Orange','TV'))
print(eucdist('Orange','Spider'))
print(eucdist('Orange','Man'))
print(eucdist('Orange','Lobster'))
print(eucdist('Orange','Boy'))


print(eucdist('Clock','Phone'))
print(eucdist('Clock','TV'))
print(eucdist('Clock','Spider'))
print(eucdist('Clock','Man'))
print(eucdist('Clock','Lobster'))
print(eucdist('Clock','Boy'))


print(eucdist('Phone','TV'))
print(eucdist('Phone','Spider'))
print(eucdist('Phone','Man'))
print(eucdist('Phone','Lobster'))
print(eucdist('Phone','Boy'))

print(eucdist('TV','Spider'))
print(eucdist('TV','Man'))
print(eucdist('TV','Lobster'))
print(eucdist('TV','Boy'))

print(eucdist('Spider','Man'))
print(eucdist('Spider','Lobster'))
print(eucdist('Spider','Boy'))

print(eucdist('Man','Lobster'))
print(eucdist('Man','Boy'))

print(eucdist('Lobster','Boy'))


3.760387659072876
4.197992324829102
3.950622320175171
3.650371789932251
3.904721260070801
4.192838191986084
4.257116317749023
4.0226545333862305
4.336188316345215
3.9166460037231445
3.698730707168579
3.909518241882324
4.175487518310547
4.443570613861084
4.085025787353516
3.874588966369629
3.695213794708252
3.8528635501861572
3.9656949043273926
4.206535339355469
3.8353381156921387
3.6174325942993164
3.92520809173584
4.330547332763672
4.380630970001221
3.9564127922058105
3.6825809478759766
3.7360780239105225
4.243814945220947
3.5815932750701904
3.657160997390747
4.016629695892334
3.6429386138916016
4.374272346496582
2.8136181831359863
4.1506428718566895


**Utility functions to add mean RGB-Euclidean Distances and Mean Edge Distances to word vectors**

In [ ]:
# For Apple
def makeVecApple(name):
  vec = retVec(name)
  vec = np.append(vec,[183.65042119898732]) 
  vec = np.append(vec,[236.8613933499185])
  vec = np.append(vec,[257.6993361970571])
  vec = np.append(vec,[155.6698040005568])
  vec = np.append(vec,[214.5706725909578])
  vec = np.append(vec,[2165.38179389155])
  vec = np.append(vec,[2479.82352747331])
  return vec
  
apple_vec = makeVecApple('Apple')
# apple_vec_fruit = makeVecApple('Fruit')
# apple_vec_device = makeVecApple('Device')

In [ ]:
# For Orange
def makeVecOrange(name):
  vec = retVec(name)
  vec = np.append(vec,[176.22351837226557]) 
  vec = np.append(vec,[239.3933294163915])
  vec = np.append(vec,[292.4997788450778])
  vec = np.append(vec,[126.13520837802974])
  vec = np.append(vec,[234.0243700434754])
  vec = np.append(vec,[2178.3295085709])
  vec = np.append(vec,[2482.0783606545])
  return vec
orange_vec = makeVecOrange('Orange')
# orange_vec_fruit = makeVecOrange('Fruit')
# orange_vec_device = makeVecOrange('Device')

In [ ]:
# For Clock
def makeVecClock(name):
  vec = retVec(name)
  vec = np.append(vec,[216.90015530847035]) 
  vec = np.append(vec,[232.7585115720062])
  vec = np.append(vec,[241.15963943447477])
  vec = np.append(vec,[179.49398677123304])
  vec = np.append(vec,[228.130332447708])
  vec = np.append(vec,[2189.39099845924])
  vec = np.append(vec,[2461.52765117746])
  return vec

clock_vec = makeVecClock('Clock')
# clock_vec_fruit = makeVecClock('Fruit')
# clock_vec_device = makeVecClock('Device')

In [ ]:
# For Phone. Telephone is termed as Phone since it is more common.
def makeVecPhone(name):
  vec = retVec(name)
  vec = np.append(vec,[228.62903329696513]) 
  vec = np.append(vec,[230.72508372210632])
  vec = np.append(vec,[233.52274961384074])
  vec = np.append(vec,[188.3947497916666])
  vec = np.append(vec,[233.23078350180631])
  vec = np.append(vec,[2232.4873857935])
  vec = np.append(vec,[2526.38915884144])
  return vec

phone_vec = makeVecPhone('Phone')
# phone_vec_fruit = makeVecPhone('Fruit')
# phone_vec_device = makeVecPhone('Device')

In [ ]:
# For TV. Television is termed as TV since it is more common.
def makeVecTV(name):
  vec = retVec(name)
  vec = np.append(vec,[214.86001948368045]) 
  vec = np.append(vec,[221.31623025032712])
  vec = np.append(vec,[224.1419148669648])
  vec = np.append(vec,[190.7617792649619])
  vec = np.append(vec,[196.75357325496464])
  vec = np.append(vec,[2213.14847160444])
  vec = np.append(vec,[2457.6942826627])
  return vec

tv_vec = makeVecTV('TV')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Spider. Television is termed as TV since it is more common.
def makeVecSpider(name):
  vec = retVec(name)
  vec = np.append(vec,[215.6829420418053]) 
  vec = np.append(vec,[215.989340981306])
  vec = np.append(vec,[240.91732373550144])
  vec = np.append(vec,[181.71494099386993])
  vec = np.append(vec,[200.7586516844691])
  vec = np.append(vec,[2301.4478454846303])
  vec = np.append(vec,[2567.731641731374])
  return vec

spider_vec = makeVecSpider('Spider')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Man. Television is termed as TV since it is more common.
def makeVecMan(name):
  vec = retVec(name)
  vec = np.append(vec,[207.09032443113122]) 
  vec = np.append(vec,[221.13045495178358])
  vec = np.append(vec,[226.2436036283648])
  vec = np.append(vec,[186.83176832089922])
  vec = np.append(vec,[188.88885708060263])
  vec = np.append(vec,[2246.33267074079])
  vec = np.append(vec,[2503.867269053288])
  return vec

man_vec = makeVecMan('Man')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Lobster. Television is termed as TV since it is more common.
def makeVecLobster(name):
  vec = retVec(name)
  vec = np.append(vec,[204.6076922096025]) 
  vec = np.append(vec,[227.85690642941998])
  vec = np.append(vec,[240.850040458154])
  vec = np.append(vec,[178.77706596480363])
  vec = np.append(vec,[202.02645253353342])
  vec = np.append(vec,[2167.986368348048])
  vec = np.append(vec,[2454.3843538300434])
  return vec

lobster_vec = makeVecLobster('Lobster')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Boy. Television is termed as TV since it is more common.
def makeVecBoy(name):
  vec = retVec(name)
  vec = np.append(vec,[204.99835704081661]) 
  vec = np.append(vec,[225.92092045701315])
  vec = np.append(vec,[235.17839848862488])
  vec = np.append(vec,[176.13910165332695])
  vec = np.append(vec,[207.44776526315897])
  vec = np.append(vec,[2222.824465116193])
  vec = np.append(vec,[2475.58971376879])
  return vec

boy_vec = makeVecBoy('Boy')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# Refined euclidean distance of two word vectors
def eucdist(wv1,wv2):
  return distance.euclidean(wv1, wv2)

In [ ]:
pd.DataFrame({'Apple':apple_vec,
 'Orange':orange_vec,
 'Clock':clock_vec,
 'Phone':phone_vec,
 'TV':tv_vec,
 'Spider':spider_vec,
 'Man':man_vec,
 'Lobster':lobster_vec,
 'Boy':boy_vec}).to_csv('Added_feat_sem_vec.csv',index = False)

In [ ]:
print(eucdist(apple_vec,orange_vec))
print(eucdist(apple_vec,clock_vec))
print(eucdist(apple_vec,phone_vec))
print(eucdist(apple_vec,tv_vec))
print(eucdist(apple_vec,spider_vec))
print(eucdist(apple_vec,man_vec))
print(eucdist(apple_vec,lobster_vec))
print(eucdist(apple_vec,boy_vec))


print(eucdist(orange_vec,clock_vec))
print(eucdist(orange_vec,phone_vec))
print(eucdist(orange_vec,tv_vec))
print(eucdist(orange_vec,spider_vec))
print(eucdist(orange_vec,man_vec))
print(eucdist(orange_vec,lobster_vec))
print(eucdist(orange_vec,boy_vec))


print(eucdist(clock_vec,phone_vec))
print(eucdist(clock_vec,tv_vec))
print(eucdist(clock_vec,spider_vec))
print(eucdist(clock_vec,man_vec))
print(eucdist(clock_vec,lobster_vec))
print(eucdist(clock_vec,boy_vec))


print(eucdist(phone_vec,tv_vec))
print(eucdist(phone_vec,spider_vec))
print(eucdist(phone_vec,man_vec))
print(eucdist(phone_vec,lobster_vec))
print(eucdist(phone_vec,boy_vec))

print(eucdist(tv_vec,spider_vec))
print(eucdist(tv_vec,man_vec))
print(eucdist(tv_vec,lobster_vec))
print(eucdist(tv_vec,boy_vec))

print(eucdist(spider_vec,man_vec))
print(eucdist(spider_vec,lobster_vec))
print(eucdist(spider_vec,boy_vec))

print(eucdist(man_vec,lobster_vec))
print(eucdist(man_vec,boy_vec))

print(eucdist(lobster_vec,boy_vec))


# print(eucdist(apple_vec,apple_vec_fruit), eucdist(apple_vec,apple_vec_device))
# print(eucdist(orange_vec,orange_vec_fruit), eucdist(orange_vec,orange_vec_device))
# print(eucdist(clock_vec,clock_vec_fruit), eucdist(clock_vec,clock_vec_device))
# print(eucdist(phone_vec,phone_vec_fruit), eucdist(phone_vec,phone_vec_device))
# print(eucdist(tv_vec,tv_vec_fruit), eucdist(tv_vec,tv_vec_device))

52.06003439199604
55.463503482781654
103.68936736079687
81.70294727540457
169.91037573431583
102.7896110053644
46.556934760804836
69.90076042486102
88.20322655398662
122.83529358268822
117.80784487297326
177.40110185396622
128.50712534743482
91.07777793398066
97.67565810730225
79.90361627912768
46.14024187350431
157.75853728099216
84.25196172148031
37.20258840007437
44.64809798071923
82.51283206410673
89.54683389453933
57.4046102050311
105.21128122070694
63.94375510594984
142.57248840765584
58.21694374735208
51.64368204359929
31.5459519923741
87.24890571432934
175.92455795350187
122.50146099744077
95.4361960210509
43.88728690243429
59.5527468279918


# **Adding word vector of RGB and Edge to targetted classes**

In [ ]:
# Returns the corresponding word vector
def retVec(word):
  return wv.get_vector(word)

# Returs euclidean distances between two word vectors
def eucdist(word1,word2):
  return distance.euclidean(retVec(word1), retVec(word2))

In [ ]:
def addVecs(name):
  vec = retVec(name)
  vec = np.append(vec,eucdist(name,'Red')) 
  vec = np.append(vec,eucdist(name,'Green')) 
  vec = np.append(vec,eucdist(name,'Blue')) 
  vec = np.append(vec,eucdist(name,'Orange')) 
  vec = np.append(vec,eucdist(name,'Black'))
  vec = np.append(vec,eucdist(name,'Circle')) 
  vec = np.append(vec,eucdist(name,'Square'))  
  return vec

In [ ]:
apple_vec = addVecs('Apple')
orange_vec = addVecs('Orange')
clock_vec = addVecs('Clock')
phone_vec = addVecs('Phone')
tv_vec = addVecs('TV')
spider_vec = addVecs('Spider')
man_vec = addVecs('Man')
lobster_vec = addVecs('Lobster')
boy_vec = addVecs('Boy')

In [ ]:
# Refined euclidean distance of two word vectors
def eucdist(wv1,wv2):
  return distance.euclidean(wv1, wv2)

In [ ]:
pd.DataFrame({'Apple':apple_vec,
 'Orange':orange_vec,
 'Clock':clock_vec,
 'Phone':phone_vec,
 'TV':tv_vec,
 'Spider':spider_vec,
 'Man':man_vec,
 'Lobster':lobster_vec,
 'Boy':boy_vec}).to_csv('Added_feat_sem_vec.csv',index = False)

In [ ]:
print(eucdist(apple_vec,orange_vec))
print(eucdist(apple_vec,clock_vec))
print(eucdist(apple_vec,phone_vec))
print(eucdist(apple_vec,tv_vec))
print(eucdist(apple_vec,spider_vec))
print(eucdist(apple_vec,man_vec))
print(eucdist(apple_vec,lobster_vec))
print(eucdist(apple_vec,boy_vec))


print(eucdist(orange_vec,clock_vec))
print(eucdist(orange_vec,phone_vec))
print(eucdist(orange_vec,tv_vec))
print(eucdist(orange_vec,spider_vec))
print(eucdist(orange_vec,man_vec))
print(eucdist(orange_vec,lobster_vec))
print(eucdist(orange_vec,boy_vec))


print(eucdist(clock_vec,phone_vec))
print(eucdist(clock_vec,tv_vec))
print(eucdist(clock_vec,spider_vec))
print(eucdist(clock_vec,man_vec))
print(eucdist(clock_vec,lobster_vec))
print(eucdist(clock_vec,boy_vec))


print(eucdist(phone_vec,tv_vec))
print(eucdist(phone_vec,spider_vec))
print(eucdist(phone_vec,man_vec))
print(eucdist(phone_vec,lobster_vec))
print(eucdist(phone_vec,boy_vec))

print(eucdist(tv_vec,spider_vec))
print(eucdist(tv_vec,man_vec))
print(eucdist(tv_vec,lobster_vec))
print(eucdist(tv_vec,boy_vec))

print(eucdist(spider_vec,man_vec))
print(eucdist(spider_vec,lobster_vec))
print(eucdist(spider_vec,boy_vec))

print(eucdist(man_vec,lobster_vec))
print(eucdist(man_vec,boy_vec))

print(eucdist(lobster_vec,boy_vec))


# print(eucdist(apple_vec,apple_vec_fruit), eucdist(apple_vec,apple_vec_device))
# print(eucdist(orange_vec,orange_vec_fruit), eucdist(orange_vec,orange_vec_device))
# print(eucdist(clock_vec,clock_vec_fruit), eucdist(clock_vec,clock_vec_device))
# print(eucdist(phone_vec,phone_vec_fruit), eucdist(phone_vec,phone_vec_device))
# print(eucdist(tv_vec,tv_vec_fruit), eucdist(tv_vec,tv_vec_device))

5.355892760598729
4.277658608902459
3.9724207707751984
3.726680603835465
4.054881883739129
4.226412675436375
4.374374257446514
4.054115999235351
6.207586631347794
5.6033598834216125
5.275905341823206
5.565480629522885
5.941290516694292
6.416477820650936
5.806277156331691
3.9087807435409783
3.8059543285061856
4.107677056433623
3.987461927737402
4.268046425480005
3.8681419573854323
3.700445064531266
4.135405722991201
4.347522042710755
4.454061686483579
3.9756155507726385
3.7826786222453457
3.801554977946631
4.4947803464965554
3.633526125965124
3.8109812747208616
4.426635943403445
3.773513042486604
4.463627039009233
2.8178483742157088
4.264355225403392


In [ ]:
'''
Content of the CIFAR 100 Dataset
0: apple

1: aquarium_fish

2: baby

3: bear

4: beaver

5: bed

6: bee

7: beetle

8: bicycle

9: bottle

10: bowl

11: boy

12: bridge

13: bus

14: butterfly

15: camel

16: can

17: castle

18: caterpillar

19: cattle

20: chair

21: chimpanzee

22: clock

23: cloud

24: cockroach

25: couch

26: cra

27: crocodile

28: cup

29: dinosaur

30: dolphin

31: elephant

32: flatfish

33: forest

34: fox

35: girl

36: hamster

37: house

38: kangaroo

39: keyboard

40: lamp

41: lawn_mower

42: leopard

43: lion

44: lizard

45: lobster

46: man

47: maple_tree

48: motorcycle

49: mountain

50: mouse

51: mushroom

52: oak_tree

53: orange

54: orchid

55: otter

56: palm_tree

57: pear

58: pickup_truck

59: pine_tree

60: plain

61: plate

62: poppy

63: porcupine

64: possum

65: rabbit

66: raccoon

67: ray

68: road

69: rocket

70: rose

71: sea

72: seal

73: shark

74: shrew

75: skunk

76: skyscraper

77: snail

78: snake

79: spider

80: squirrel

81: streetcar

82: sunflower

83: sweet_pepper

84: table

85: tank

86: telephone

87: television

88: tiger

89: tractor

90: train

91: trout

92: tulip

93: turtle

94: wardrobe

95: whale

96: willow_tree

97: wolf

98: woman

99: worm
'''